In [1]:
from akita_utils.dna_utils import hot1_rc, dna_1hot
import numpy as np
import pandas as pd
import pysam

2022-12-16 04:01:33.574779: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home1/smaruj/software/GSL/lib:/home1/smaruj/software/HTSLIB/lib
2022-12-16 04:01:33.574820: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
from akita_utils.seq_gens import _insert_casette

In [3]:
from basenji import seqnn, stream, dna_io

In [4]:
background_file = "/project/fudenber_735/tensorflow_models/akita/v2/analysis/background_seqs.fa"

In [5]:
genome_fasta = "/project/fudenber_735/genomes/mm10/mm10.fa"

In [ ]:
genome_open = pysam.Fastafile(genome_fasta)

In [ ]:
seq_coords_df = pd.read_csv("./test.tsv", sep="\t")

In [ ]:
background_seqs = []
with open(background_file, "r") as f:
    for line in f.readlines():
        if ">" in line:
            continue
        background_seqs.append(dna_io.dna_1hot(line.strip()))
num_insert_backgrounds = seq_coords_df["background_index"].max()
if len(background_seqs) < num_insert_backgrounds:
    raise ValueError(
        "must provide a background file with at least as many"
        + "backgrounds as those specified in the insert seq_coords tsv."
        + "\nThe provided background file has {len(background_seqs)} sequences."
    )

In [ ]:
seq_coords_df = seq_coords_df.head(500).tail(10)

In [ ]:
seq_coords_df

In [ ]:
def split_concat_to_integers(concat_string):
    return [int(i) for i in concat_string.split(",")]

In [ ]:
def split_concat_to_strings(concat_string):
    return [i for i in concat_string.split(",")]

In [ ]:
12 + 19 + 12

In [ ]:
def fixed_core_insertion_seqs_gen(seq_coords_df, background_seqs, genome_open)
    
    for s in seq_coords_df.itertuples():
    
        flank_bp = s.flank_bp
        spacer_bp = s.spacer_bp
        orientation_string = s.orientation

        core_chrom, flank_chrom = split_concat_to_strings(s.chrom)
        core_start, flank_start = split_concat_to_integers(s.start)
        core_end, flank_end = split_concat_to_integers(s.end)
        core_strand, flank_strand = split_concat_to_strings(s.strand)

        # getting flank motif
        seq_1hot_flank = dna_1hot(genome_open.fetch(flank_chrom, flank_start - flank_bp, flank_end + flank_bp).upper())
        if flank_strand == "-":
            seq_1hot_flank = hot1_rc(seq_1hot_flank)

        seq_1hot_flank_left = seq_1hot_flank[:flank_bp,:]
        seq_1hot_flank_right = seq_1hot_flank[-flank_bp:,:]

        # getting core motif
        seq_1hot_core = dna_1hot(genome_open.fetch(core_chrom, core_start, core_end).upper())
        if core_strand == "-":
            seq_1hot_core = hot1_rc(seq_1hot_core)

        # joining all the chunks together
        seq_1hot_insertion = np.concatenate((seq_1hot_flank_left, seq_1hot_core, seq_1hot_flank_right), axis=0)

        seq_1hot = background_seqs[s.background_index].copy()

        seq_1hot = _insert_casette(seq_1hot, seq_1hot_insertion, spacer_bp, orientation_string)

        yield seq_1hot
    